In [8]:
import pandas
import numpy
import os
from dataaccess.csv import get_data

df = get_data( 'G$_prices.csv' )

print(df.tail())

        Date  Open  High  Low   Close  Volume  Adjustment
4 2023-10-20   NaN   NaN  NaN  198850     NaN           1
3 2023-10-23   NaN   NaN  NaN  197300     NaN           1
2 2023-10-24   NaN   NaN  NaN  196365     NaN           1
1 2023-10-25   NaN   NaN  NaN  198330     NaN           1
0 2023-10-26   NaN   NaN  NaN  197500     NaN           1


In [9]:
from indicators.average import rolling_average, wilder_average

from trading.trade import Trade
from trading.signal import cross_from_below

from plotly.subplots import make_subplots
import plotly.graph_objects as go


def strategy(params, dates, closes, average_func):
    short_ma = average_func(closes, params['short_ma'])
    long_ma = average_func(closes, params['long_ma'])
    ultra_long_ma = average_func(closes, params['ultra_long_ma'])
    
    crosses = cross_from_below(short_ma, long_ma)
    above_ultra_long = closes - ultra_long_ma
    
    live_deals = []
    dead_deals = []

    for i, (date, close, cross, above) in enumerate(zip( dates, closes, crosses, above_ultra_long)):
        if cross and above > 0:
            live_deals.append(Trade(date, close, 1, close*(1 - params['stop']), close*(1 + params['take_profit']), dates[min(i + params['holding_days'],len(dates)-1)] ))
        
        for d in live_deals:
            if close <= d.stop or close >= d.take_profit or date >= d.expiry:
                d.set_exit_price(close)
                dead_deals.append(d)
                live_deals.remove(d)
        
    for d in live_deals:
        d.set_exit_price(closes[-1])
        dead_deals += d

    return dead_deals

def compute_daily_pandl(deals, dates, closes):
    
    pandls = numpy.zeros(len(dates))
    for i, (date, close) in enumerate(zip(dates, closes)):
       
        for d in deals:
            if d.date == date:
                pandls[i] += close - d.traded_price
            elif date > d.date and date < d.expiry:
                pandls[i] += close - closes[i-1]
            elif d.expiry == date:
                pandls[i] += d.exit_price - closes[i-1]
    return pandls



params = dict(short_ma = 13, long_ma = 26, ultra_long_ma = 200, stop = 0.1, take_profit = 0.1, holding_days = 13)

deals = strategy(params, df['Date'].values, df['Close'].values, rolling_average)
daily_pandls = compute_daily_pandl(deals, df['Date'].values, df['Close'].values )

cum_pandl = numpy.cumsum(daily_pandls)

win_pct = sum([1 for d in deals if d.exit_price - d.traded_price > 0])/len(deals)
pandl= sum([d.exit_price - d.traded_price for d in deals])
losses = [d.exit_price - d.traded_price for d in deals if (d.exit_price - d.traded_price) <= 0] 
wins = [d.exit_price - d.traded_price for d in deals if (d.exit_price - d.traded_price) > 0]

p_and_l_ratio = (sum(wins)/len(wins))/(sum(numpy.abs(losses))/len(losses)) if len(losses) else 1

print(f'P&L {pandl:.2f}')
print(f'DailyPandLs {sum(daily_pandls)}')
print(f'Deals {len(deals)}')
print(f"Win Pct {100*win_pct:.2f}")
print(f"Pts per deal {pandl/len(deals):.2f}")
print(f"P&L Ratio {p_and_l_ratio}")
print(f"Return {win_pct*(1+p_and_l_ratio)}")

annual_return = 262*numpy.mean(daily_pandls)
vol = pow(262,0.5)*numpy.std(daily_pandls)
print('Sharpe Ratio: ', annual_return/vol)




fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.0)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.0,row_heights=[0.8,0.2])

fig.add_trace(go.Scatter(x=df.Date.values, y=df.Close.values, name='Close'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.Date.values, y=cum_pandl, name='Cum PandL'), row=2, col=1 )



P&L 50814.00
DailyPandLs 50814.0
Deals 76
Win Pct 57.89
Pts per deal 668.61
P&L Ratio 1.085148040679494
Return 1.207190970919707
Sharpe Ratio:  0.2842883287176035


In [10]:
deals_df = pandas.DataFrame.from_dict([Trade.to_dict(d) for d in deals])
results_path = os.environ.get('RESULTS_PATH')
deals_df.to_csv(os.path.join(results_path,'oil.csv'),mode='w')

In [11]:
from indicators.rsi import rsi

srs = numpy.array([  54.8, 56.8, 57.85, 59.85, 60.57, 61.1, 62.17, 60.60
        , 62.35, 62.15, 62.35, 61.45, 62.8, 61.37, 62.5, 62.57
        , 60.8, 59.37, 60.35, 62.35, 62.17, 62.55, 64.55
        , 64.37, 65.30, 64.42, 62.90, 61.60, 62.05, 60.05, 59.70
        , 60.90, 60.25, 58.27, 58.70, 57.72, 58.10, 58.20])

n = 14

rsi_srs = rsi(srs, n)

In [12]:
import unittests.trading.test_trailing_stop
import unittests.trading.test_cross_from_below

unittests.trading.test_trailing_stop.test_trailing_stop()
unittests.trading.test_cross_from_below.test_cross_from_below()






In [13]:
import plotly

plotly.plot(rsi_srs, kind='line')


In [14]:
from indicators.true_range import true_range, atr

opens = [52.8, 52.6, 52.0, 52.2, 52.10, 51.9, 51.5, 51.15, 51.50 ]
highs = [53, 52.75, 52.35, 52.45, 52.35, 52.1, 51.8, 51.6, 51.7 ]
lows = [52.5, 52.25, 51.85, 52.15, 51.75, 51.50, 51.00, 51.25, 51.40]
closes = [52.7, 52.55, 52.3, 52.4, 51.90, 51.65, 51.10, 51.55, 51.65 ]

print(len(opens), len(highs), len(lows), len(closes) )

tr = true_range(highs, lows, closes)

print(tr)

print(atr(highs, lows, closes, 7))


9 9 9 9
[0.5  0.5  0.7  0.3  0.65 0.6  0.8  0.5  0.3 ]
[       nan        nan        nan        nan        nan        nan
 0.57857143 0.56734694 0.52915452]
